In [1]:
#%matplotlib inline
%matplotlib widget
from astropy.cosmology import LambdaCDM
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import astropy.units as u
from scipy.integrate import quad
import ezgal # BC03 model maker
import os

# Setup Cosmology

In [2]:
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7, Tcmb0=2.725)

# Create Stellar Population

In [3]:
# check to make sure we have defined the bpz filter path
if not os.getenv('EZGAL_FILTERS'):
    os.environ['EZGAL_FILTERS'] = (f'{os.environ["HOME"]}/Projects/planckClusters/MOSAICpipe/bpz-1.99.3/FILTER/')

model = ezgal.model('bc03_ssp_z_0.02_salp.model')
model = model.make_exponential(1)
model.set_cosmology(Om=cosmo.Om0, Ol=cosmo.Ode0, h=cosmo.h, w=cosmo.w(0))
                                   
model.add_filter('g_MOSAICII.res', name='g')
model.add_filter('r_MOSAICII.res', name='r')
model.add_filter('i_MOSAICII.res', name='i')
model.add_filter('z_MOSAICII.res', name='z')
model.add_filter('K_KittPeak.res', name='K')

# Blanton 2003 Normalization
Mr_star = -20.44 + 5 * np.log10(cosmo.h) # abs mag.
# set the normalization
model.set_normalization('sloan_r', 0.1, Mr_star, vega=False)                                   

/home/boada/.local/lib/python3.6/site-packages/ezgal/data/models/bc03_ssp_z_0.02_salp.model


/home/boada/.local/lib/python3.6/site-packages/scipy/integrate/quadrature.py:358: RuntimeWarning: divide by zero encountered in true_divide
  h0divh1 = h0 / h1
/home/boada/.local/lib/python3.6/site-packages/scipy/integrate/quadrature.py:360: RuntimeWarning: divide by zero encountered in true_divide
  y[slice1]*hsum*hsum/hprod +
/home/boada/.local/lib/python3.6/site-packages/scipy/integrate/quadrature.py:361: RuntimeWarning: invalid value encountered in add
  y[slice2]*(2-h0divh1))


# Calculate a few things to get going.

In [4]:
# desired formation redshift
zf = 6.0
# fetch an array of redshifts out to given formation redshift
zs = model.get_zs(zf)
 
# Calculate some cosmological stuff
DM = cosmo.distmod(zs)
dlum = cosmo.luminosity_distance(zs)

# Define the functions that we'll need

Need to compute the cluster volume...

$M_{vir} = 4/3 \pi r^3_{vir} \rho_c(r<r_{vir}) = 4/3 \pi r^3_{vir} \Delta_c \rho_c$

if we let $\Delta_c = 200$ then 

$M_{200} = 4/3 \pi r^3_{200} 200 \rho_c$ with $\rho_c = \frac{3H(z)^2}{8\pi G}$

or just $M_{200} = V_{200}200\rho_c$. So we'll make a function to calculate $\rho_c$. And we'll make use of the astropy units package to do all the unit analysis for us.

Don't forget that $H(z) = H_0E(z)$ 

## We also need to integrate the Schechter luminosity functions..

The Schechter Function:

For Luminosity:

$\Phi(L) = \phi^\star \frac{L}{L_\star}^\alpha e^{-\frac{L}{L_\star}}$

For Magnitudes:

$\Phi(M) = \phi^\star\frac{2}{5}log(10) (10^{\frac{2}{5}(M_\star - M)})^{\alpha+1} e^{-10^{\frac{2}{5}(M_\star - M)}}$




In [37]:
def rho_crit(z, cosmo):
    # convert G into better units:
    G = const.G.to(u.km**2 * u.Mpc/(u.M_sun * u.s**2))
    return 3 / (8 * np.pi * G) * cosmo.H0**2 * cosmo.efunc(z)**2 # Mpc^3

def schechterL(luminosity, phiStar, alpha, LStar): 
    """Schechter luminosity function.""" 
    LOverLStar = (luminosity/LStar) 
    return (phiStar/LStar) * LOverLStar**alpha * np.exp(- LOverLStar) 

def schechterM(magnitude, phiStar, alpha, MStar): 
    """Schechter luminosity function by magnitudes.""" 
    MStarMinM = 0.4 * (MStar - magnitude)
    return (0.4 * np.log(10) * phiStar * 10.0**(MStarMinM * (alpha + 1.)) * np.exp(-10.**MStarMinM))

# Mass limits from PSZ2

In [28]:
from astropy.table import Table
from scipy.interpolate import interp1d

z1 = 0
z2 = 2
dz = 0.025

# build the mass array
zarr = np.arange(z1, z2 + dz, dz)

ps2 = Table.read('../../catalogs/PSZ2v1.fits')
df2 = ps2.to_pandas()
data = df2[['REDSHIFT', 'MSZ']]
data['REDSHIFT'].replace(-1, np.nan, inplace=True)

# redshift bins
zbins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 3]

nMasses = 100
big_mass = []
for j in range(nMasses):
    mass = np.ones_like(zarr) * 1e14
    for i in range(len(zbins) - 1):
        mask = (zbins[i] <= zarr) & (zarr < zbins[i + 1])
        mass[mask] *= float(data.loc[(zbins[i] <= data['REDSHIFT']) & (data['REDSHIFT'] < zbins[i + 1]), 'MSZ'].sample()) * cosmo.h
    big_mass.append(mass)

mass = np.vstack(big_mass)
mass_func = interp1d(zarr, np.median(mass, axis=0))

/home/boada/.local/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


# Start Calculating things

In [44]:
# So now we are going to calculate the volumes as a function of z

#M200 = mass_func(zarr) * u.solMass

M200 = 1e15 * u.solMass
V200 = M200/ (200 * rho_crit(zs, cosmo))

# Calculate the M_star values
Mstar = model.get_absolute_mags(zf, filters='i', zs=zs)

# calculate the abs mag of our limiting magnitude as a function of z
mlim = 23.5
#Mlim = Mstar - 2.5 * np.log10(0.4)
Mlim = mlim - cosmo.distmod(zs).value - model.get_kcorrects(zf, filters='i', zs=zs)

# Here are the Schechter function stuff from Liu et al.
phi_star = 3.6 * cosmo.efunc(zs)**2
alpha = -1.05 * (1 + zs)**(-2/3)
fr = 0.8*(1 + zs)**(-1/2)

#alpha = np.ones_like(alpha) * -1
#Mpiv = 6e14 * u.solMass
#zpiv = 0.6

#alpha = -0.96 * (M200 / Mpiv)**0.01 * ((1 + zs)/ (1 + zpiv))**-0.94
#phi_star = 1.68 * (M200 / Mpiv)**0.09 * ((1 + zs)/ (1 + zpiv))**0.09 * cosmo.efunc(zs)**2
#fr = 0.62 * (M200 / Mpiv)**0.08 * ((1 + zs)/ (1 + zpiv))** -0.80


In [45]:
LF = []
for phi, a, M_star, M_lim in zip(phi_star, alpha, Mstar, Mlim):
    if M_lim < M_star - 2.5 * np.log10(0.4):
        Mlimit = M_lim
    else:
        Mlimit = M_star - 2.5 * np.log10(0.4)
    y, err = quad(schechterM, -np.inf, Mlimit, args=(phi, a, M_star))
    #print(M_star - M_lim, y)
    LF.append(y)

/home/boada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in double_scalars
  
/home/boada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/home/boada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  import sys


In [46]:
plt.figure()
plt.plot(zs, (LF * V200.value + 1) * fr)
ax = plt.gca()
ax.set_yticks(np.arange(0, 75, 10))
plt.xlim(0.1, 5)
plt.ylim(0, 80)
plt.xlabel('redshift')
plt.ylabel('N (r < r200)')
plt.grid()

FigureCanvasNbAgg()

In [18]:
# calculate the abs mag of our limiting magnitude as a function of z
mlim = 23.5
#Mlim = model.get_absolute_mags(zf, filters='i', zs=zs) - 2.5 * np.log10(0.4)
Mlim = mlim - cosmo.distmod(zs).value - model.get_kcorrects(zf, filters='i', zs=zs)
plt.figure()
plt.plot(zs, model.get_absolute_mags(zf, filters='i', zs=zs))
plt.plot(zs, Mlim)
plt.plot(zs, model.get_absolute_mags(zf, filters='i', zs=zs) - 2.5 * np.log10(0.4))
plt.grid()

FigureCanvasNbAgg()

In [13]:
Mlim

array([ -8.1658134 ,  -9.68495902, -10.57712692, -11.21590893,
       -11.71317736, -12.12290307, -12.47160961, -12.77411805,
       -13.04159075, -13.28498939, -13.50334666, -13.7055737 ,
       -13.89190301, -14.06351244, -14.22556981, -14.37699888,
       -14.5193843 , -14.65682555, -14.78502903, -14.9069075 ,
       -15.45391931, -15.91286253, -16.29646466, -16.62851574,
       -16.93172572, -17.20374221, -17.45777789, -17.69273943,
       -17.91992918, -18.12987769, -18.32865681, -18.51828907,
       -18.69976994, -18.86891746, -19.03642649, -19.19397521,
       -19.34179258, -19.4892439 , -19.63047416, -19.77424126,
       -19.91108518, -20.03864248, -20.16951124, -20.29877324,
       -20.42751841, -20.55921112, -20.69559866, -20.83252436,
       -20.96665018, -21.09965252, -21.22894866, -21.35956163,
       -21.47795764, -21.59165994, -21.69933654, -21.80210694,
       -21.9075376 , -22.00622275, -22.10366806, -22.20027099,
       -22.29100864, -22.36640173, -22.42845094, -22.48

In [12]:
Mstar - 2.5 * np.log10(0.4)

array([-20.57008079, -20.57597325, -20.58179183, -20.5875379 ,
       -20.59315221, -20.59866339, -20.60410763, -20.60948613,
       -20.61491286, -20.62028775, -20.62559825, -20.63085243,
       -20.6361888 , -20.64146161, -20.64667197, -20.65183576,
       -20.65708687, -20.66227606, -20.66740441, -20.67247647,
       -20.69788217, -20.72276832, -20.74717636, -20.77114779,
       -20.79472324, -20.81792192, -20.84079074, -20.86338788,
       -20.88574247, -20.90787483, -20.92980046, -20.95167168,
       -20.97363203, -20.99554516, -21.01738616, -21.03918731,
       -21.06161016, -21.08403246, -21.10630224, -21.12975616,
       -21.1531245 , -21.17677915, -21.20122247, -21.2250567 ,
       -21.24963478, -21.27334722, -21.29763075, -21.32132178,
       -21.34592426, -21.36980908, -21.39509567, -21.4191294 ,
       -21.44521967, -21.47012112, -21.49621011, -21.52236259,
       -21.54762003, -21.57405979, -21.59920148, -21.62422419,
       -21.64926965, -21.67312985, -21.69756533, -21.72

In [13]:
np.array(LF)

array([1.36011857, 1.36379449, 1.36755501, 1.37139974, 1.37532829,
       1.37934029, 1.38343539, 1.38761326, 1.39187359, 1.39621607,
       1.40064043, 1.40514639, 1.4097337 , 1.41440212, 1.41915142,
       1.4239814 , 1.42889184, 1.43388257, 1.43895341, 1.44410419,
       1.47105233, 1.49998075, 1.53088041, 1.56374745, 1.59858256,
       1.63539041, 1.67417914, 1.71495994, 1.75774672, 1.80255577,
       1.8494055 , 1.89831621, 1.94930989, 2.00241005, 2.05764154,
       2.11503043, 2.17460392, 2.23639018, 2.30041828, 2.36671813,
       2.43532036, 2.50625632, 2.57955794, 2.65525776, 2.73338882,
       2.81398464, 2.8970792 , 2.98270688, 3.07090245, 3.16170102,
       3.25513804, 3.35124925, 3.36398578, 3.22797693, 3.107751  ,
       2.99752963, 2.87481824, 2.7701184 , 2.66237721, 2.55378918,
       2.45820885, 2.39777082, 2.3723643 , 2.35913165, 2.34543234,
       2.33164145, 2.34507622, 2.3544848 , 2.35308712, 2.38667178,
       2.42581826, 2.45644373, 2.48928701, 2.53582618, 2.57234

In [14]:
alpha

<Quantity [-1.49391678, -1.48696386, -1.48007739, -1.47325642, -1.46650001,
           -1.45980724, -1.4531772 , -1.44660901, -1.44010179, -1.4336547 ,
           -1.42726689, -1.42093755, -1.41466586, -1.40845103, -1.4022923 ,
           -1.39618888, -1.39014003, -1.38414503, -1.37820313, -1.37231364,
           -1.34362827, -1.31615347, -1.28981347, -1.26453872, -1.24026526,
           -1.21693419, -1.19449113, -1.17288587, -1.15207189, -1.13200609,
           -1.11264845, -1.09396178, -1.07591146, -1.0584652 , -1.04159289,
           -1.02526639, -1.00945939, -0.99414723, -0.97930682, -0.96491647,
           -0.95095582, -0.93740571, -0.92424813, -0.91146609, -0.89904359,
           -0.88696552, -0.87521761, -0.86378638, -0.85265905, -0.84182357,
           -0.83126847, -0.82098291, -0.8109566 , -0.80117976, -0.7916431 ,
           -0.78233779, -0.77325543, -0.76438803, -0.75572796, -0.74726797,
           -0.73900114, -0.73092085, -0.72302081, -0.71529499, -0.70773763,
           -

In [15]:
Phi

NameError: name 'Phi' is not defined

In [ ]:
np.exp(-1)